In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  4 15:11:48 2018

@author: Fang
"""

import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy import create_engine
import os
import datetime as dt
import time
import urllib.request as urlreq
import json

module_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\Modules\\DataCollection'
os.chdir(module_dir)


def hist_prices(ticker, start_date, end_date = dt.datetime.today()):
    start_date_unix = int(start_date.timestamp())
    end_date = dt.datetime.today()
    end_date_unix = int(end_date.timestamp())

    hist_price_url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&period1={1}&period2={2}&interval=1d'.format(ticker,start_date_unix,end_date_unix)
    #print(hist_price_url)
    with urlreq.urlopen(hist_price_url) as url:
        data = json.loads(url.read().decode())
        hist_prices = pd.DataFrame({**data['chart']['result'][0]['indicators']['quote'][0], **data['chart']['result'][0]['indicators']['adjclose'][0]},
                                        index = [dt.datetime.utcfromtimestamp(int(x)).date() for x in 
                                                 data['chart']['result'][0]['timestamp']])
        hist_prices.index = pd.to_datetime(hist_prices.index)
    
    return hist_prices

start_date = dt.datetime(1999,1,1)

# Initializing Stock Universe
os.chdir('..\\')
os.chdir('..\\')
os.chdir('..\\Data\\Stock Universe')
#tsx = pd.read_csv('TSX.csv')['Symbol']#.tolist()
nyse = pd.read_csv('NYSE.csv')['Symbol']#.tolist()
nasdaq = pd.read_csv('NASDAQ.csv')['Symbol']#.tolist()
amex = pd.read_csv('AMEX.csv')['Symbol']#.tolist()
us_names = pd.concat([nyse,nasdaq,amex]).drop_duplicates().values.tolist()

del nyse, nasdaq, amex

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)

prices_db_dir = 'D:\\Price Data'
os.chdir(prices_db_dir)
hisprices_engine = create_engine('sqlite:///histprices.db', echo = False)

run_update = True

def return_profiles(ticker):
    start_time = time.time()
    prices_query = 'SELECT Date, adjclose FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
    curr_closes = pd.read_sql_query(prices_query, con=hisprices_engine, index_col = 'Date')
    
    if len(curr_closes) == 0:
        return (ticker,)*8
    curr_closes.index = pd.to_datetime(curr_closes.index)

    yahoo_db_query = 'SELECT Underlying, industry, sector FROM profiles WHERE Underlying = "{}"'.format(ticker)
    curr_yahoo_db = pd.read_sql_query(yahoo_db_query, con = yahoo_engine, index_col = 'Underlying').reset_index().drop_duplicates('Underlying').reset_index(drop = True)
    if len(curr_yahoo_db) == 0:
        curr_yahoo_db = pd.DataFrame({'Underlying': [ticker], 'industry': [np.nan], 'sector': [np.nan]})
        
    quarter_mapping = {1: 1, 2:1, 3:1, 
                       4:2, 5:2, 6:2,
                       7:3, 8:3, 9:3,
                       10:4, 11:4, 12:4}

    curr_closes['year'] = curr_closes.index.year
    curr_closes['month'] = curr_closes.index.month
    curr_closes['boy'] = curr_closes['year'] - curr_closes['year'].shift(1)
    curr_closes['bom'] = curr_closes['month'] - curr_closes['month'].shift(1)
    curr_closes['boq'] = curr_closes['month'] % 3
    curr_closes['quarter'] = curr_closes['month'].replace(quarter_mapping)
    curr_closes['Q'] = curr_closes['year'].apply(lambda x: str(x)) + ' Q' + curr_closes['quarter'].apply(lambda x: str(x))
    curr_closes['dailyReturns'] = curr_closes['adjclose'].pct_change()

    annual_returns = curr_closes[curr_closes['boy'] != 0][['adjclose','year']]
    annual_returns['adjclose'] = annual_returns['adjclose'].pct_change().shift(-1)
    annual_returns = annual_returns.set_index('year').dropna()
    annual_returns.columns = ['return']

    quarterly_returns = curr_closes[curr_closes['bom'] != 0][['adjclose','boq','Q']]
    quarterly_returns = quarterly_returns[quarterly_returns['boq'] == 1][['adjclose','Q']]
    quarterly_returns['adjclose'] = quarterly_returns['adjclose'].pct_change().shift(-1)
    quarterly_returns = quarterly_returns.set_index('Q').dropna()
    quarterly_returns.columns = ['return']

    def risk_profiles(returns_df, curr_closes, curr_yahoo_db):
        maxDrawDowns = pd.DataFrame(index = returns_df.index, columns = ['maxDrawDowns'])
        downsideVols = pd.DataFrame(index = returns_df.index, columns = ['downsideVols'])
        returnVols = pd.DataFrame(index = returns_df.index, columns = ['vol'])

        for period in returns_df.index:
            curr_year = curr_closes[curr_closes[returns_df.index.name] == period][['adjclose','dailyReturns']]
            curr_year['cumReturns'] = (curr_year['dailyReturns'] + 1).cumprod()
            curr_year['rollingMax'] = curr_year['cumReturns'].expanding(min_periods=1).max()
            curr_year['drawDown'] = curr_year['cumReturns']/curr_year['rollingMax'] - 1
            curr_downside = curr_year[curr_year['dailyReturns'] < 0]

            maxDrawDowns.loc[period,'maxDrawDowns'] = curr_year['drawDown'].min()
            downsideVols.loc[period,'downsideVols'] = curr_downside['dailyReturns'].std()*np.sqrt(len(curr_downside))
            returnVols.loc[period, 'vol'] = curr_year['dailyReturns'].std()*np.sqrt(252)

        maxDrawDowns, downsideVols, returnVols = maxDrawDowns.T, downsideVols.T, returnVols.T
        maxDrawDowns.index, downsideVols.index, returnVols.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values
        return  maxDrawDowns, downsideVols, returnVols

    annualDDs, annualDownsides, annualVols = risk_profiles(annual_returns, curr_closes, curr_yahoo_db)
    quarterlyDDs, quarterlyDownsides, quarterlyVols = risk_profiles(quarterly_returns, curr_closes, curr_yahoo_db)
    annual_returns, quarterly_returns = annual_returns.T, quarterly_returns.T
    annual_returns.index, quarterly_returns.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values

    print("Completed in %s seconds" % (time.time() - start_time))
    
    return annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols



if run_update:
    
    start_time = time.time()
    i = 0
    total_length = len(us_names)
    for ticker in us_names:
        

        try:
            curr_hist_prices = hist_prices(ticker, start_date)
            curr_hist_prices = curr_hist_prices.sort_index(ascending = True)
            curr_hist_prices['Underlying'] = ticker
        except:
            print('Failed prices for {}'.format(ticker))
            print('{0:.2f}% Completed'.format(i/total_length*100))
            continue

        table_in_db = True

        try:
            query = 'SELECT Date, Underlying FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
            curr_hist_db = pd.read_sql_query(query, con=hisprices_engine, index_col = 'Date').tail(1)
            curr_hist_db.index = pd.to_datetime(curr_hist_db.index)
        except:
            table_in_db = False

        if table_in_db and len(curr_hist_db) != 0:
            latest_date_in_db = curr_hist_db.index.tolist()[0]
            curr_hist_prices = curr_hist_prices[curr_hist_prices.index > latest_date_in_db]

        curr_hist_prices.to_sql('historicalPrices', con=hisprices_engine, 
                                if_exists='append', index_label = 'Date')
        i += 1
        print('{0:.2f}% Completed'.format(i/total_length*100))

    print("Completed in %s seconds" % (time.time() - start_time))

0.01% Completed
0.02% Completed
0.03% Completed
0.05% Completed
0.06% Completed
0.07% Completed
0.08% Completed
0.09% Completed
0.10% Completed
0.12% Completed
0.13% Completed
0.14% Completed
0.15% Completed
0.16% Completed
0.17% Completed
Failed prices for ABR-A
0.17% Completed
Failed prices for ABR-B
0.17% Completed
Failed prices for ABR-C
0.17% Completed
0.18% Completed
0.20% Completed
0.21% Completed
0.22% Completed
0.23% Completed
0.24% Completed
0.25% Completed
0.27% Completed
0.28% Completed
0.29% Completed
0.30% Completed
0.31% Completed
0.32% Completed
0.33% Completed
0.35% Completed
0.36% Completed
0.37% Completed
0.38% Completed
0.39% Completed
0.40% Completed
0.42% Completed
0.43% Completed
0.44% Completed
0.45% Completed
0.46% Completed
0.47% Completed
0.48% Completed
0.50% Completed
0.51% Completed
0.52% Completed
0.53% Completed
0.54% Completed
0.55% Completed
0.57% Completed
0.58% Completed
0.59% Completed
0.60% Completed
Failed prices for AFS-A
0.60% Completed
Failed p

3.57% Completed
3.58% Completed
3.59% Completed
3.60% Completed
3.61% Completed
3.62% Completed
3.64% Completed
3.65% Completed
3.66% Completed
3.67% Completed
3.68% Completed
3.69% Completed
3.71% Completed
3.72% Completed
3.73% Completed
3.74% Completed
3.75% Completed
3.76% Completed
3.77% Completed
3.79% Completed
Failed prices for BML-G
3.79% Completed
Failed prices for BML-H
3.79% Completed
Failed prices for BML-J
3.79% Completed
Failed prices for BML-L
3.79% Completed
3.80% Completed
3.81% Completed
3.82% Completed
3.83% Completed
3.84% Completed
3.86% Completed
3.87% Completed
3.88% Completed
3.89% Completed
3.90% Completed
3.91% Completed
3.92% Completed
3.94% Completed
3.95% Completed
3.96% Completed
3.97% Completed
3.98% Completed
3.99% Completed
4.01% Completed
4.02% Completed
4.03% Completed
Failed prices for BRK.A
4.03% Completed
Failed prices for BRK.B
4.03% Completed
4.04% Completed
4.05% Completed
4.06% Completed
4.07% Completed
4.09% Completed
4.10% Completed
4.11% Co

Failed prices for DDR-J
7.47% Completed
Failed prices for DDR-K
7.47% Completed
7.48% Completed
7.49% Completed
7.50% Completed
7.51% Completed
7.53% Completed
7.54% Completed
7.55% Completed
7.56% Completed
7.57% Completed
7.58% Completed
7.60% Completed
7.61% Completed
7.62% Completed
7.63% Completed
7.64% Completed
Failed prices for DHC.X
7.64% Completed
7.65% Completed
7.66% Completed
7.68% Completed
7.69% Completed
7.70% Completed
7.71% Completed
7.72% Completed
7.73% Completed
7.75% Completed
7.76% Completed
7.77% Completed
7.78% Completed
7.79% Completed
7.80% Completed
7.81% Completed
Failed prices for DLN-A
7.81% Completed
7.83% Completed
7.84% Completed
7.85% Completed
Failed prices for DLR-C
7.85% Completed
Failed prices for DLR-G
7.85% Completed
Failed prices for DLR-H
7.85% Completed
Failed prices for DLR-I
7.85% Completed
Failed prices for DLR-J
7.85% Completed
7.86% Completed
7.87% Completed
7.88% Completed
7.90% Completed
7.91% Completed
7.92% Completed
7.93% Completed


In [9]:
# spx = hist_prices('^GSPC', start_date)
# spx = spx.sort_index(ascending = True)
# spx['Underlying'] = 'SPX'
# spx = spx[['Underlying','adjclose']]
# spx['industry'] = 'SP500'
# spx['sector'] = 'Index'

In [ ]:
start_time = time.time()
i = 0
total_length = len(us_names)
return_engine = create_engine('sqlite:///returnProfiles.db', echo = False)

for ticker in us_names:
    annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols = return_profiles(ticker)
    if type(annual_returns) != str and ticker != 'AAPL':
        annual_returns.to_sql('annualReturns', con=return_engine, 
                              if_exists='append', index_label = 'Underlying')
        annualDDs.to_sql('annualDDs', con=return_engine, 
                         if_exists='append', index_label = 'Underlying')
        annualDownsides.to_sql('annualDownsides', con=return_engine, 
                               if_exists='append', index_label = 'Underlying')
        annualVols.to_sql('annualVols', con=return_engine, 
                          if_exists='append', index_label = 'Underlying')
        
        quarterly_returns.to_sql('quarterlyReturns', con=return_engine, 
                                 if_exists='append', index_label = 'Underlying')
        quarterlyDDs.to_sql('quarterlyDDs', con=return_engine, 
                            if_exists='append', index_label = 'Underlying')
        quarterlyDownsides.to_sql('quarterlyDownsides', con=return_engine, 
                                  if_exists='append', index_label = 'Underlying')
        quarterlyVols.to_sql('quarterlyVols', con=return_engine, 
                             if_exists='append', index_label = 'Underlying')    
    i += 1
    print('{0:.2f}% Completed'.format(i/total_length*100))

print("Completed in %s seconds" % (time.time() - start_time))